<a href="https://colab.research.google.com/github/Yannan-Chen/allenVisualBehaviorOphys/blob/main/dataLoader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# @title Install packages
!pip install pip --upgrade --quiet
!pip install mindscope_utilities --quiet
!pip install pandas --quiet
!pip install seaborn --quiet

In [3]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.manifold import TSNE

import mindscope_utilities
import mindscope_utilities.visual_behavior_ophys as ophys

from allensdk.brain_observatory.behavior.behavior_project_cache import VisualBehaviorOphysProjectCache

pd.set_option('display.max_columns', 500)

In [4]:
data_storage_directory = "/temp"  # Note: this path must exist on your local drive
cache = VisualBehaviorOphysProjectCache.from_s3_cache(cache_dir=data_storage_directory)

ophys_session_table.csv: 100%|██████████| 227k/227k [00:00<00:00, 1.14MMB/s] 
behavior_session_table.csv: 100%|██████████| 1.21M/1.21M [00:00<00:00, 5.29MMB/s]
ophys_experiment_table.csv: 100%|██████████| 610k/610k [00:00<00:00, 2.96MMB/s] 
ophys_cells_table.csv: 100%|██████████| 4.29M/4.29M [00:00<00:00, 8.94MMB/s]


In [5]:
session_table = cache.get_ophys_session_table()
experiment_table = cache.get_ophys_experiment_table()

In [19]:
session_sst = session_table[['Sst' in session_table.loc[i].driver_line[0] for i in session_table.index]]
session_slc = session_table[['Slc' in session_table.loc[i].driver_line[0] for i in session_table.index]]
session_vip = session_table[['Vip' in session_table.loc[i].driver_line[0] for i in session_table.index]]

print('SST', len(session_sst))
print('Slc', len(session_slc))
print('Vip', len(session_vip))

SST 158
Slc 361
Vip 186


In [20]:
ophys_session_id = 854060305


In [21]:
neural_data = []
ophys_experiment_ids = session_table.loc[ophys_session_id]['ophys_experiment_id']
for ophys_experiment_id in ophys_experiment_ids:
  this_experiment = cache.get_behavior_ophys_experiment(ophys_experiment_id)

  this_experiment_neural_data = ophys.build_tidy_cell_df(this_experiment)

  # add some columns with metadata for the experiment
  metadata_keys = [
      'ophys_experiment_id',
      'ophys_session_id',
      'targeted_structure',
      'imaging_depth',
      'equipment_name',
      'cre_line',
      'mouse_id',
      'sex',
  ]
  for metadata_key in metadata_keys:
    this_experiment_neural_data[metadata_key] = this_experiment.metadata[metadata_key]

  # append the data for this experiment to a list
  neural_data.append(this_experiment_neural_data)

# concatate the list of dataframes into a single dataframe
neural_data = pd.concat(neural_data)


behavior_ophys_experiment_854759890.nwb: 100%|██████████| 233M/233M [00:15<00:00, 15.4MMB/s]
behavior_ophys_experiment_854759894.nwb: 100%|██████████| 255M/255M [00:16<00:00, 15.7MMB/s]
behavior_ophys_experiment_854759896.nwb: 100%|██████████| 240M/240M [00:16<00:00, 14.8MMB/s]
behavior_ophys_experiment_854759898.nwb: 100%|██████████| 259M/259M [00:17<00:00, 14.5MMB/s]
behavior_ophys_experiment_854759900.nwb: 100%|██████████| 242M/242M [00:15<00:00, 15.6MMB/s]
behavior_ophys_experiment_854759903.nwb: 100%|██████████| 254M/254M [00:17<00:00, 14.7MMB/s]
